In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import warnings
from datetime import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
warnings.simplefilter("ignore")

In [ ]:
# 加载数据集
data = pd.read_csv('data33.csv')

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.PATIENT_UPDATE.value_counts()

In [ ]:
data=data.astype(int)
data=data.loc[:,['PROBABLE_DIAGNOSE','DYSPNEA','DIAGNOSE_OF_PNEUMONIA','POLYPNEA','INTUBATION','CHEST_PAIN',
                 'FATIGUE_LOSS_APPETITE','SUDDEN_ONSET_OF_SYMPTOMS','MYALGIA','COUGH','FINAL_RESULT','FEVER','TYPE_OF_PATIENT','ARTHRALGIA',
                 'SEX', 'ODYNOPHAGIA','IRRITABILITY','DIARRHEA','CHILLS','CEPHALEA','RHINORRHEA','VOMIT'
                 ,'ABDOMINAL_PAIN','CONJUNCTIVITIS','CYANOSIS','DIABETES','EPOC' ,'ASTHMA','IMMUNOSUPPRESSION','HYPERTENSION',
               'HIV','CARDIOVASCULAR_DISEASE','OBESITY','CHRONIC_KIDNEY_DISEASE','SMOKER','PATIENT_UPDATE']]
data

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, Dropout, MaxPooling1D, Flatten, SimpleRNN, BatchNormalization
def teacher_model():
    model = Sequential()
    model.add(SimpleRNN(512,input_shape=(35,1)))
    model.add(Activation("relu"))
    model.add(Dropout(0.25))
    model.add(BatchNormalization())
    model.add((Flatten()))
    model.add(Dense(2048, activation="relu"))
    model.add(Dense(1024, activation="relu"))
    model.add(Dense(512, activation="relu"))
    model.add(Dense(256, activation="relu"))
    model.add(Dense(128, activation="relu"))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(2, activation="softmax")) 
    model.summary()
    model.summary()
    return model

In [ ]:
def student_model():
    model = Sequential()
    model.add(SimpleRNN(512,input_shape=(10,1)))
    model.add(Activation("relu"))
    model.add(Dropout(0.25))
    model.add(BatchNormalization())
    model.add((Flatten()))
    model.add(Dense(1024, activation="relu"))
    model.add(Dense(512, activation="relu"))
    model.add(Dense(256, activation="relu"))

    model.add(Dense(2, activation="softmax")) 
    model.summary()
    model.summary()
    return model

In [ ]:
teachermodel=teacher_model()

In [ ]:
from sklearn.model_selection import train_test_split
X = data.drop(["PATIENT_UPDATE"], axis = 1)
Y = data["PATIENT_UPDATE"]
x_train,x_test,y_train,y_test= train_test_split(X,Y,test_size=0.3)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
x_train

In [ ]:
x1_train=x_train.iloc[:,:10]
x1_test=x_test.iloc[:,:10]
x1_train

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#resize the input data
x_train = np.array(x_train).reshape(3989,35,1)
x_test = np.array(x_test).reshape(1710,35,1)

In [ ]:
from tensorflow.keras.optimizers import SGD
opt = SGD(lr=0.1, decay=0.1, momentum=0.2, nesterov=True)

In [ ]:
teachermodel.compile(loss="binary_crossentropy", optimizer = opt, metrics=["accuracy"])

In [ ]:
teachermodel.fit(x_train, y_train,batch_size=100, epochs = 100, validation_data=(x_test,y_test))

In [ ]:
x_train_float32 = tf.cast(x_train, tf.float32)
teacherout=teachermodel.predict((x_train_float32))

In [ ]:
teacherout

In [ ]:
y_train

In [ ]:
studentmodel=student_model()

In [ ]:
studentmodel.compile(loss="binary_crossentropy", optimizer = opt, metrics=["accuracy"])

In [ ]:
x1_train = np.array(x1_train).reshape(3989,10,1)
x1_test = np.array(x1_test).reshape(1710,10,1)

In [ ]:
studentmodel.fit(x1_train, y_train,batch_size=100, epochs = 100, validation_data=(x1_test,y_test))

In [ ]:
t_out = teacherout
model = studentmodel
for l in model.layers:
        l.trainable = True     
    
label_test = y_test
model.compile(loss="binary_crossentropy", optimizer = opt, metrics=["accuracy"])
model.fit(x1_train,t_out ,batch_size=100, epochs = 100, validation_data=(x1_test,y_test))

In [ ]:
y_test

In [ ]:
x1_test_float32 = tf.cast(x1_test, tf.float32)
y_score = model.predict((x1_test_float32))

In [ ]:
y_pred=np.around(y_score[:, 1])
y_test=y_test[:, 1]

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_pred)
precision, recall, f1 = precision_recall_fscore_support(y_test,y_pred,average='macro')[:-1]
print("accuracy: ", accuracy)
print("precision: ",precision)
print("recall: ",recall)
print("f1: ",f1)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
import seaborn as sns
def print_confusionMatrix(Y_TestLabels, PredictedLabels):
    confusionMatx = confusion_matrix(Y_TestLabels, PredictedLabels)
    
    precision = confusionMatx/confusionMatx.sum(axis = 0)
    
    recall = (confusionMatx.T/confusionMatx.sum(axis = 1)).T
    
    sns.set(font_scale=1.5)
    
    # confusionMatx = [[1, 2],
    #                  [3, 4]]
    # confusionMatx.T = [[1, 3],
    #                   [2, 4]]
    # confusionMatx.sum(axis = 1)  axis=0 corresponds to columns and axis=1 corresponds to rows in two diamensional array
    # confusionMatx.sum(axix =1) = [[3, 7]]
    # (confusionMatx.T)/(confusionMatx.sum(axis=1)) = [[1/3, 3/7]
    #                                                  [2/3, 4/7]]

    # (confusionMatx.T)/(confusionMatx.sum(axis=1)).T = [[1/3, 2/3]
    #                                                    [3/7, 4/7]]
    # sum of row elements = 1
    
    labels = ["0","1"]

    
    plt.figure(figsize=(18,10),dpi=60)
    sns.heatmap(confusionMatx, cmap = "YlGnBu_r", annot = True, fmt = ".1f", xticklabels=labels, yticklabels=labels)
    plt.title("FDT(RNN) Confusion Matrix", fontsize = 30)
    plt.xlabel('Predicted Class', fontsize = 30)
    plt.ylabel('Original Class', fontsize = 30)
    plt.tick_params(labelsize = 25)
    plt.xticks(rotation = 90)
    plt.savefig("Pictures/FDT(RNN)-Confusion Matrix.pdf")
    plt.savefig("Pictures/FDT(RNN)-Confusion Matrix.png")
    plt.show()

    
    print("-"*125)
    
    plt.figure(figsize=(18,10),dpi=60)
    sns.heatmap(precision, cmap = "YlGnBu_r", annot = True, fmt = ".2f", xticklabels=labels, yticklabels=labels)
    plt.title("FDT(RNN) Precision Matrix", fontsize = 30)
    plt.xlabel('Predicted Class', fontsize = 30)
    plt.ylabel('Original Class', fontsize = 30)
    plt.tick_params(labelsize = 25)
    plt.xticks(rotation = 90)
    plt.savefig("Pictures/FDT(RNN)-Precision Matrix.pdf")
    plt.savefig("Pictures/FDT(RNN)-Precision Matrix.png")
    plt.show()
    print("-"*125)
    
    plt.figure(figsize=(18,10),dpi=60)
    sns.heatmap(recall, cmap = "YlGnBu_r", annot = True, fmt = ".2f", xticklabels=labels, yticklabels=labels)
    plt.title("FDT(RNN) Recall Matrix", fontsize = 30)
    plt.xlabel('Predicted Class', fontsize = 30)
    plt.ylabel('Original Class', fontsize = 30)
    plt.tick_params(labelsize = 25)
    plt.xticks(rotation = 90)
    plt.savefig("Pictures/FDT(RNN)-Recall Matrix.pdf")
    plt.savefig("Pictures/FDT(RNN)-Recall Matrix.png")
    plt.show()
    sensitivity = confusionMatx[0,0]/(confusionMatx[0,0]+confusionMatx[1,0])
    print('Sensitivity : ', sensitivity )

    specificity = confusionMatx[1,1]/(confusionMatx[1,1]+confusionMatx[0,1])
    print('Specificity : ', specificity)
    
def plot_roc_curve(fpr, tpr):
    plt.figure()
    lw = 2
    roc_auc = auc(fpr,tpr)
    plt.plot(fpr, tpr, color='darkorange',lw=lw, 
             label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0,1], [0,1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0, 1.0])
    plt.ylim([0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    #plt.title('')
    plt.legend(loc="lower right")
    plt.show()  

In [ ]:
print_confusionMatrix(y_test, y_pred)

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, _ = roc_curve(y_test, y_score[:,1])
plot_roc_curve(fpr, tpr)    

In [ ]:
df1 = pd.DataFrame({'y_test':y_test,'y_score[:,1]': y_score[:,1]})
df1.to_csv('Pictures/pre_lab_15.csv',index=False)